In [ ]:
# используемые библиотеки
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from math import sqrt
import statsmodels.api as sm
pd.options.mode.chained_assignment = None

In [204]:
# обработка данных
data = pd.read_excel('C:/Users/Dmitry/Desktop/JN/University_studies/Rashetka/Data/Data.xlsx').dropna()
data = data.transpose()
data.columns = data.iloc[2]
data = data.drop(index = (['Country Code','Series Name','Series Code','Country Name']))
USA_data = data.iloc[:,:6]
NOR_data = data.iloc[:,6:]
N = len(USA_data.index)

In [212]:
# переобозначение переменных и создание пронумерованного временного ряда
for i in USA_data,NOR_data:
    i.columns = ['Y','C','I','G','Im','Ex']
for i in USA_data,NOR_data:
    for j in i.columns:
        i[j] = i[j].astype(float)
x_quad = np.concatenate((np.array(list(range(1,N+1))).reshape((-1,1)),np.array([i**2 for i in range(1,N+1)]).reshape((-1,1))),axis = 1)
x_lin = np.array(list(range(1,N+1))).reshape((-1,1))

In [213]:
# выбор страны и переход к абсолютным величинам
df = NOR_data.copy()
for j in df.columns[1:]:
    df.loc[:,j] = (df.loc[:,j]/100)*df.loc[:,'Y']

In [214]:
# создание функции которая запоминает ряд ошибок регрессии
def get_dev(y,lin):
    if lin == 1:
        result_lin = pd.DataFrame(columns = ['Linear','Fact','Predict','Dev'])
        result_lin['Fact'] = y
        result_lin['Linear'] = x_lin.reshape(-1,1)
        model = LinearRegression().fit(x_lin,y)
        result_lin['Predict'] = result_lin['Linear']*model.coef_ + model.intercept_
        result_lin['Dev'] = result_lin['Fact'] - result_lin['Predict']
        return result_lin['Dev']
    else:
        result = pd.DataFrame(columns = ['Linear','Quad','Fact','Predict','Dev'])
        result['Fact'] = y
        result[['Linear','Quad']] = x_quad.reshape(-1,2)
        model = LinearRegression().fit(x_quad,y)
        result['Predict'] = result['Linear']*model.coef_[0] + result['Quad']*model.coef_[1] + model.intercept_
        result['Dev'] = result['Fact'] - result['Predict']
        return result['Dev']

In [215]:
# заполнение очищенных от тренда данных по переменным для разных способов выделения тренда
cycle_lin = pd.DataFrame(columns = ['Y','C','G','I','Ex','Im','TB/Y','G/Y','TB','(X+M)/Y'],index = df.index, dtype = float)
cycle_quad = pd.DataFrame(columns = ['Y','C','G','I','Ex','Im','TB/Y','G/Y','TB','(X+M)/Y'],index = df.index, dtype = float)
cycle_hp100 = pd.DataFrame(columns = ['Y','C','G','I','Ex','Im','TB/Y','G/Y','TB','(X+M)/Y'],index = df.index, dtype = float)
cycle_hp6_25 = pd.DataFrame(columns = ['Y','C','G','I','Ex','Im','TB/Y','G/Y','TB','(X+M)/Y'],index = df.index, dtype = float)

for i in ['Y','C','G','I','Ex','Im']:
    y = np.log(df.loc[:,i])
    cycle_lin.loc[:,i] = get_dev(y,1)
    cycle_quad.loc[:,i] = get_dev(y,0)
    cycle_hp100.loc[:,i] = sm.tsa.filters.hpfilter(y, 100)[0]
    cycle_hp6_25.loc[:,i] = sm.tsa.filters.hpfilter(y, 6.25)[0]
    
y_tby = (df.loc[:,'Ex'] - df.loc[:,'Im'])/df.loc[:,'Y']
cycle_lin.loc[:,'TB/Y'], cycle_quad.loc[:,'TB/Y'], cycle_hp100.loc[:,'TB/Y'], cycle_hp6_25.loc[:,'TB/Y']= get_dev(y_tby,1), get_dev(y_tby,0), sm.tsa.filters.hpfilter(y_tby, 100)[0], sm.tsa.filters.hpfilter(y_tby, 6.25)[0]

for i in [cycle_lin,cycle_quad,cycle_hp100,cycle_hp6_25]:
    i.loc[:,'G/Y'] = i.loc[:,'G']/i.loc[:,'Y']

for i in [cycle_lin,cycle_quad,cycle_hp100,cycle_hp6_25]:
    i.loc[:,'TB'] = i.loc[:,'Ex'] - i.loc[:,'Im']


for i in [cycle_lin,cycle_quad,cycle_hp100,cycle_hp6_25]:
    i.loc[:,'(X+M)/Y'] = (i.loc[:,'Ex'] + i.loc[:,'Im'])/i.loc[:,'Y']

In [216]:
stds = pd.DataFrame(columns = ['lin','quad','hp100','hp6.25'],index = ['Y','C','G','I','Ex','Im','TB/Y'])
corrs = pd.DataFrame(columns = ['lin','quad','hp100','hp6.25'],index = ['Y','C','G/Y','I','Ex','Im','TB/Y','TB'])
scorrs = pd.DataFrame(columns = ['lin','quad','hp100','hp6.25'],index = ['Y','C','G','I','Ex','Im','TB/Y'])
means = pd.DataFrame(columns = ['lin','quad','hp100','hp6.25'],index = ['TB/Y','(X+M)/Y'])

for i,elem in enumerate(['Y','C','G','I','Ex','Im','TB/Y']):
    if elem == 'Y' or elem == 'TB/Y':
        stds.iloc[i,0] = sqrt(N/(N-2))*cycle_lin[elem].std(ddof=0)
        stds.iloc[i,1] = sqrt(N/(N-3))*cycle_quad[elem].std(ddof=0)
        stds.iloc[i,2] = cycle_hp100[elem].std(ddof=0)
        stds.iloc[i,3] = cycle_hp6_25[elem].std(ddof=0)
    else:
        stds.iloc[i,0] = sqrt(N/(N-2))*cycle_lin[elem].std(ddof=0)/stds.iloc[0,0]
        stds.iloc[i,1] = sqrt(N/(N-3))*cycle_quad[elem].std(ddof=0)/stds.iloc[0,1]
        stds.iloc[i,2] = cycle_hp100[elem].std(ddof=0)/stds.iloc[0,2]
        stds.iloc[i,3] = cycle_hp6_25[elem].std(ddof=0)/stds.iloc[0,3]

for i,elem in enumerate(['Y','C','G/Y','I','Ex','Im','TB/Y','TB']):
    for h,k in enumerate([cycle_lin,cycle_quad,cycle_hp100,cycle_hp6_25]):
        corrs.iloc[i,h] = k[elem].corr(k['Y'],method = 'pearson')

for i,elem in enumerate(['Y','C','G','I','Ex','Im','TB/Y']):
    for h,k in enumerate([cycle_lin,cycle_quad,cycle_hp100,cycle_hp6_25]):
        scorrs.iloc[i,h] = sm.tsa.acf(k[elem],nlags = 2)[1]

for i,elem in enumerate(['TB/Y','(X+M)/Y']):
    for h,k in enumerate([cycle_lin,cycle_quad,cycle_hp100,cycle_hp6_25]):
        means.iloc[i,h] = k[elem].mean()

In [217]:
stds.index = ['δy','δc/δy','δg/δy','δi/δy','δx/δy','δm/δy','δtb/y']

In [218]:
with pd.ExcelWriter('C:/Users/Dmitry/Desktop/JN/University_studies/Rashetka/Output/Norway.xlsx') as writer:  
    stds.to_excel(writer, sheet_name='Standard Deviations')
    corrs.to_excel(writer, sheet_name='Correlations with y')
    scorrs.to_excel(writer, sheet_name='Serial Correlations')
    means.to_excel(writer, sheet_name='Means')